In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

import numpy as np

Using TensorFlow backend.


In [2]:
##Minkowski product of 4-vectors p1, p2.
def m_prod_arr(p1, p2):
    return np.multiply(p1[:,0], p2[:,0]) - np.sum(np.multiply(p1[:,1:], p2[:,1:]), axis=1)

In [3]:
def data_preprocessing(me_filename, mom_filename, frac=1):
    ##Data Aquisition
    me_raw = np.load(me_filename, allow_pickle=True) #Matrix elements
    mom_raw = np.load(mom_filename, allow_pickle=True, encoding='bytes') #4-momenta of inputs
    mom_raw = np.array([np.array(element) for element in mom_raw])
    
    me_raw=me_raw[:int(frac*len(me_raw))]
    mom_raw=mom_raw[:int(frac*len(mom_raw))]
    
    p_12 = mom_raw[:,2] + mom_raw[:,3] #p1+p2
    p_13 = mom_raw[:,2] + mom_raw[:,4] #p1+p3
    p_23 = mom_raw[:,3] + mom_raw[:,4] #p2+p3

    s_12 = m_prod_arr(p_12, p_12) #center of mass energies of two of three beams
    s_13 = m_prod_arr(p_13, p_13)
    s_23 = m_prod_arr(p_23, p_23)
    
    ##Flatten Momentum
    mom = np.array([np.ndarray.flatten(np.array(element)[2:]) for element in mom_raw])

    ##Reformat Matrix Element (remove divergent behaviour)
    me = np.multiply(np.multiply(me_raw, s_13), s_23)/(s_12+s_13+s_23)
    
    return(me, mom)

In [4]:
##Obtain & process data
me_train, mom_train = data_preprocessing('LO_3_MG/3Jet_3000000.npy', 'LO_3_0.01_NJet/PS3_0.01_3000000.npy', frac=0.1)
me_test, mom_test = data_preprocessing('LO_3_MG/3Jet_500000.npy', 'LO_3_0.01_NJet/PS3_0.01_500000.npy', frac=0.1)

##Rescale
scaler = StandardScaler()

mom_train = scaler.fit_transform(mom_train) #Rescale on training set
mom_test = scaler.transform(mom_test) #Rescale on test set

In [5]:
# Random Model from Tensorflow Website
def build_model(dropout, hidden_layers=5, components=200, optimizer='RMSprop'):
    
  model = keras.Sequential([
    keras.layers.Dense(components, activation='relu', input_shape=[12]), #could try swish??
    keras.layers.BatchNormalization(),  
    keras.layers.Dropout(dropout),
      
    *np.repeat([keras.layers.Dense(components, activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout)], hidden_layers),
    
    keras.layers.Dense(components, activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout),
    
    keras.layers.Dense(1) 
  ])

  model.compile(loss='mse', #gans?
                optimizer=optimizer,
                metrics=['mse'])
  return model

In [6]:
model = KerasRegressor(build_fn=build_model, epochs=3)

dropout = [0.3,0.4,0.5]
param_grid = dict(dropout=dropout)

grid = RandomizedSearchCV(model, param_grid, cv=3)
grid_result = grid.fit(mom_train[:100000], me_train[:100000])

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/3
66666/66666 [==============================]66666/66666 [==============================] - 19s 286us/step - loss: 0.1088 - mean_squared_error: 0.1088

Epoch 2/3
66666/66666 [==============================]66666/66666 [==============================] - 19s 281us/step - loss: 3.0033e-04 - mean_squared_error: 3.0033e-04

Epoch 3/3
66666/66666 [==============================]66666/66666 [==============================] - 18s 276us/step - loss: 1.2177e-04 - mean_squared_error: 1.2177e-04

33334/33334 [==============================]33334/33334 [==============================] - 2s 75us/step

Epoch 1/3
66667/66667 [==============================]66667/66667 [==============================] - 20s 293us/step - loss: 0.1202 - mean_squared_error: 0.1202

Epoch 2/3
66667/66667 [==============================]66667/66667 [==============================] - 19s 282us/step - loss: 2.9066e-04 - mean_squared_error: 2.906

In [7]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -149201.496120 using {'dropout': 0.5}
-68515008.965965 (68301991.291308) with: {'dropout': 0.3}
-14806972.801880 (5951164.275980) with: {'dropout': 0.4}
-149201.496120 (104714.456035) with: {'dropout': 0.5}


In [6]:
#model = build_model(grid_result.best_params_['dropout'])
model = build_model(0.5)
model.fit(mom_train, me_train, epochs=3)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/3
300000/300000 [==============================]300000/300000 [==============================] - 71s 236us/step - loss: 0.0501 - mean_squared_error: 0.0501

Epoch 2/3
300000/300000 [==============================]300000/300000 [==============================] - 71s 237us/step - loss: 1.4324e-04 - mean_squared_error: 1.4324e-04

Epoch 3/3
300000/300000 [==============================]300000/300000 [==============================] - 72s 239us/step - loss: 1.2855e-04 - mean_squared_error: 1.2855e-04



In [8]:
model.evaluate(mom_test, me_test, batch_size=64, verbose=1)

50000/50000 [==============================]50000/50000 [==============================] - 2s 37us/step



[6344488.30976, 6344488.30976]

In [9]:
print(model.metrics_names)

['loss', 'mean_squared_error']


In [13]:
print(np.mean(me_train))
print(np.std(me_train))

0.06921789967290297
0.024785264292307374


In [14]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(100*np.divide(np.abs((y_true - y_pred),y_true)))

In [ ]:
me_predict = model.predict(mom_test)

print(np.mean(100*np.divide(np.abs(me_test - me_predict), me_test)))

In [ ]:
print(model.predict(mom_test[:10]))
print(me_test[:10])